In [13]:
from IPython.display import Image
import ee
ee.Initialize()
import sys 
sys.path.append('/rheil/notebooks/pygee')
import hcs_database as hcs_db
import cloud_mask
from IPython.display import display
import numpy as np

In [14]:
%%javascript
require.undef('geemap-widget');

define('geemap-widget', ["jupyter-js-widgets",'jquery', 'underscore'], function(widgets, $, _) {
    /**
     * A simple model to represent a layer on the map.
     *
     * @constructor
     */
    var Layer = Backbone.Model.extend({
        defaults: function() {
            return {
                config: {},
                type: undefined,
                visible: true
            };
        }
    });



    /**
     * A collection of layers.
     *
     * @constructor
     */
    var LayerCollection = Backbone.Collection.extend({
        model: Layer
    });



    /**
     * Override of the main widget model to intercept messages from Python
     * update Javascript state correctly.
     *
     * @constructor
     */
    var GoogleMapsModel = widgets.WidgetModel.extend({
        
        defaults: _.extend({}, widgets.DOMWidgetModel.prototype.defaults, {
            _model_name : 'GoogleMapsModel',
            _view_name : 'GoogleMapsView',
            _model_module : 'geemap-widget',
            _view_module : 'geemap-widget'
        }),
        
        /** @override */
        initialize: function() {
            this.listenTo(
                this, 'msg:custom', _.bind(this.handleMessage, this));
            // this.set('layers', new LayerCollection());
        },

        /**
         * Handle a message from Python.
         *
         * @param {!Object} payload Payload of the message.
         */
        handleMessage: function(payload) {
            if (!this.get('layers')) {
                this.set('layers', new LayerCollection());
            }
            switch(payload.action) {
                case 'addLayer':
                    this.get('layers').add({
                        config: payload.config,
                        type: payload.type
                    });
                    break;
                case 'removeLayer':
                    console.error('removeLayer not implemented');
                    break;
            }
        }
    });

    /**
     * A Google Maps API widget.
     *
     * @constructor
     */
    var GoogleMapsView = widgets.DOMWidgetView.extend({
        /**
         * Load the Maps API JS if needed, also prepare a deferred in case any
         * map methods are called before the map is ready.
         */
        initialize: function() {
            // Deferred to track for when the map is ready.
            this.mapsReadyDeferred = $.Deferred();

            // Dynamically adding Google Maps API JS here. Using a deferred to
            // track its load status as require returns as soon as the first
            // script loads and the Maps API triggers more scripts to append
            // which leaves a race condition where require thinks Maps API JS
            // is ready when it is not yet.
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            // Another instance of this view may have already loaded the Maps
            // API JS, do not try to load it twice.
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                };
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'],
                    function() {},
                    function() {});
            }
        },

        /**
         * Render the map.
         */
        render: function() {
            // We must wait until the Maps API JS is ready.
            $.when(this.mapsDeferred.promise()).then(_.bind(function() {
                // Empty the views DOM. There seem to be some weird side
                // effects when you render more than one instance of this view
                // in the notebook. Cleaning the view DOM and deferring the map
                // initialization seems to work around this. It seesm almost if
                // map instances are sharing some DOM somehow.
                this.$el.empty();
                _.defer(_.bind(function() {
                    this.$map = $([
                        '<div style="height: ',
                        this.model.get('height'),
                        '; width: ',
                        this.model.get('width'),
                        ';"></div>'
                    ].join(''));
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {
                            lat: this.model.get('lat'),
                            lng: this.model.get('lng')
                        },
                        zoom: this.model.get('zoom')
                    });

                    // Notify the map when the map container changes size via
                    // the exposed properties in the model.
                    this.listenTo(
                        this.model, 'change:height', _.bind(function() {
                            this.$map.height(this.model.get('height'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));
                    this.listenTo(
                        this.model, 'change:width', _.bind(function() {
                            this.$map.width(this.model.get('width'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));

                    // Bind a change in the position of the map to the model.
                    google.maps.event.addListener(
                        this.map,
                        'bounds_changed',
                        _.bind(this.syncFromMap, this));

                    // Bind a change in the model (coming from the Python side)
                    // to the location of the map.
                    this.listenTo(
                        this.model,
                        'change:lat',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:lng',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:zoom',
                        _.bind(this.syncFromModel, this));

                    // Render the initial set of layers.
                    if (!this.model.get('layers')) {
                        this.model.set('layers', new LayerCollection());
                    }
                    this.model.get('layers').each(this.buildLayer, this);

                    // Bind to changes in the layers of the model to stay in
                    // sync.
                    this.listenTo(
                        this.model.get('layers'),
                        'add',
                        _.bind(this.buildLayer, this));
                    this.listenTo(
                        this.model.get('layers'),
                        'remove',
                        function() {
                            console.error('removeLayer not implemented');
                        });

                    // Even though a google.maps.Map instance should be ready
                    // immediately, it is not. This delay lets the stack clear
                    // and initial map bounds to be set.
                    _.delay(_.bind(function() {
                        this.mapsReadyDeferred.resolve();
                    }, this), 500);
                }, this));
            }, this));
        },

        /**
         * Sync the values from the map into the model.
         */
        syncFromMap: function() {
            this.model.set({
                lat: this.map.getCenter().lat(),
                lng: this.map.getCenter().lng(),
                zoom: this.map.getZoom()
            });
            // This is needed for the model to update the equivalent properties
            // on the Python instance of this view.
            this.model.save_changes();
        },

        /**
         * Move the map to match the values from the model.
         */
        syncFromModel: function() {
            this.map.setCenter(new google.maps.LatLng(
                this.model.get('lat'),
                this.model.get('lng')
            ));
            this.map.setZoom(this.model.get('zoom'));
        },

        /**
         * Add a layer to the map based on its model.
         *
         * @param {!Layer} layer The layer to add.
         */
        buildLayer: function(layer) {
            switch(layer.get('type')) {
                case 'geojsondata':
                    this.addGeoJsonLayer(layer.get('config').data);
                    break;
                case 'geojsonurl':
                    this.loadGeoJsonLayer(layer.get('config').url);
                    break;
                case 'kmlurl':
                    this.loadKmlLayer(layer.get('config').url);
                    break;
                case 'earthengine':
                    this.addEarthEngineLayer(
                        layer.get('config').mapid, layer.get('config').token);
                    break;
            }
        },

        /**
         * Adds GeoJSON to the map.
         *
         * @param {!Object} data A GeoJSON object.
         */
        addGeoJsonLayer: function(data) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.addGeoJson(data);
            }, this));
        },

        /**
         * Adds a URL location of GeoJSON to the map.
         *
         * @param {string} url The URL of the GeoJSON file to load.
         */
        loadGeoJsonLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                this.map.data.loadGeoJson(url);
            }, this));
        },

        /**
         * Add a KML layer to the map.
         *
         * @param {string} url The URL of the KML file to load.
         */
        loadKmlLayer: function(url) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                new google.maps.KmlLayer({
                    url: url,
                    map: this.map
                });
            }, this));
        },

        /**
         * Add an Earth Engine layer to the map.
         *
         * @param {string} mapid The id of the Earth Engine layer.
         * @param {string} token The OAuth token to authenticate with.
         */
        addEarthEngineLayer: function(mapid, token) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                var eeMapOptions = {
                    getTileUrl: function(tile, zoom) {
                        var url = [
                            'https://earthengine.googleapis.com/map',
                            mapid,
                            zoom,
                            tile.x,
                            tile.y
                        ].join('/');
                        url += '?token=' + token;
                        return url;
                    },
                    tileSize: new window.google.maps.Size(256, 256),
                    opacity: 1.0,
                };

                // Create the overlay map type.
                var mapType = new window.google.maps.ImageMapType(eeMapOptions);

                // Overlay the Earth Engine generated layer.
                this.map.overlayMapTypes.push(mapType);
            }, this));
        }
    });

    return {
        GoogleMapsModel: GoogleMapsModel,
        GoogleMapsView: GoogleMapsView
    }
});


<IPython.core.display.Javascript object>

In [15]:
from ipywidgets import widgets
import traitlets

class GoogleMapsView(widgets.DOMWidget):
    """Google Maps API widget."""
    _model_name = traitlets.Unicode('GoogleMapsModel').tag(sync=True)
    _view_name = traitlets.Unicode('GoogleMapsView').tag(sync=True)
    _view_module = traitlets.Unicode('geemap-widget').tag(sync=True)
    _model_module = traitlets.Unicode('geemap-widget').tag(sync=True)
    lat = traitlets.CFloat(0).tag(sync=True)
    lng = traitlets.CFloat(0).tag(sync=True)
    zoom = traitlets.CInt(2).tag(sync=True)
    height = traitlets.CUnicode('300px').tag(sync=True)
    width = traitlets.CUnicode('400px').tag(sync=True)
    layers = traitlets.List([]).tag(sync=False)

    def addGeoJsonLayer(self, data):
        """Adds a dictionary of GeoJSON to the map.

        NOTE: It is likely if you are using a third party GeoJSON library you
        will have to first serialize the data into a simple dictionary before
        passing the data to this method.

        Args:
            data: A simple python dictionary of GeoJSON data.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsondata',
            'config': {'data': data}
        })

    def loadGeoJsonLayer(self, url):
        """Adds a URL location of GeoJSON to the map.

        Args:
            url: The URL of the GeoJSON file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'geojsonurl',
            'config': {'url': url}
        })

    def loadKmlLayer(self, url):
        """Adds a KML layer to the map.

        Args:
            url: The URL of the KML file.
        """
        self.send({
            'action': 'addLayer',
            'type': 'kmlurl',
            'config': {'url': url}
        })

    def addEarthEngineLayer(self, image, vis_params):
        """Adds an Earth Engine layer to the map.

        Args:
            image: The ee.Image to display.
            vis_params: Dictionary of visualization parameters.
        """
        mapid = image.getMapId(vis_params)
        self.send({
            'action': 'addLayer',
            'type': 'earthengine',
            'config': {
                'mapid': mapid['mapid'],
                'token': mapid['token']
            }
        })


In [16]:
strata_map = ee.FeatureCollection(hcs_db.maps_dict['gar_pgm'])
study_area = strata_map.geometry().bounds()
coords = study_area.getInfo()
center_lng = np.mean([coords['coordinates'][0][0][0], coords['coordinates'][0][1][0]])
center_lat = np.mean([coords['coordinates'][0][0][1], coords['coordinates'][0][2][1]])
center = ee.Geometry.Point([center_lat, center_lng])


In [17]:
input_classes = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])
output_classes = ee.List([1,1,1,2,2,0,0,0,0,0,0,1,0,0,0,0])
strata_map = strata_map.remap(lookupIn = input_classes, lookupOut = output_classes, columnName = 'code_int')

In [18]:
strata_img = strata_map.reduceToImage(
    properties = ['code_int'],
    reducer = ee.Reducer.first())

strata_img = strata_img.select(['first'], ['strata'])

Prep spectral data - Landsat 8

In [19]:
ic = ee.ImageCollection('LANDSAT/LC8_L1T_TOA_FMASK')
ic = ic.filterDate('2010-01-01', '2016-06-01')
ic = ic.filterMetadata(name = 'WRS_ROW', operator = 'less_than', value = 120)
ic = ic.filterBounds(study_area)
raw_img = ee.Image(ic.first())
ic_prepper = cloud_mask.prep_ic('l8', 'toa_fmask')
ic_masked = ic.map(ic_prepper)
clean_img = ic_masked.qualityMosaic('ndvi')

Switch to different sensor?

In [20]:
# ic = ee.ImageCollection('LANDSAT/LT5_L1T_TOA_FMASK')
# ic = ic.filterDate('2010-01-01', '2016-06-01')
# ic = ic.filterBounds(study_area)
# raw_img = ee.Image(ic.first())
# ic_prepper = cloud_mask.prep_ic('l5', 'toa_fmask')
# ic_masked = ic.map(ic_prepper)
# clean_img = ic_masked.qualityMosaic('ndvi')

Sample data

In [21]:
training = clean_img.addBands(strata_img).sample(numPixels = 5000, seed = 0, scale = 30, 
                                                 region = study_area)
classifier = ee.Classifier.cart().train(training, 'strata')

Accuracy of classifier on training data

In [ ]:
classified = clean_img.classify(classifier)
trainAccuracy = classifier.confusionMatrix()
print(trainAccuracy.accuracy().getInfo())
trainAccuracy.getInfo()

Out of sample accuracy

In [ ]:
validation = clean_img.addBands(strata_img).sample(numPixels = 1000, seed = 1, scale = 30, region = study_area)
validated = validation.classify(classifier)
testAccuracy = validated.errorMatrix('strata', 'classification')
print(testAccuracy.accuracy().getInfo())
testAccuracy.getInfo()

In [ ]:
# Create map widgets.
map_a = GoogleMapsView(lng = center_lng, lat = center_lat, zoom = 11, height = '250px', width = '800px')
map_b = GoogleMapsView(height = map_a.height, width = map_a.width)
map_c = GoogleMapsView(height = map_a.height, width = map_a.width)

# Link the map widgets.
widgets.jslink((map_a, 'lat'), (map_b, 'lat'))
widgets.jslink((map_a, 'lng'), (map_b, 'lng'))
widgets.jslink((map_a, 'zoom'), (map_b, 'zoom'))

widgets.jslink((map_a, 'lat'), (map_c, 'lat'))
widgets.jslink((map_a, 'lng'), (map_c, 'lng'))
widgets.jslink((map_a, 'zoom'), (map_c, 'zoom'))

# Add the layers to the maps
map_a.addEarthEngineLayer(image=clean_img.select(['nir', 'swir1', 'red']), vis_params={'min': 0, 'max': .6})
map_b.addEarthEngineLayer(image=strata_img, vis_params={'min': 0, 'max': 10})
map_c.addEarthEngineLayer(image=classified, vis_params={'min': 0, 'max': 10})

# Display
display(map_a)
display(map_b)
display(map_c)